In [ ]:
!pip install transformers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import Dataset, DataLoader
import os
from tqdm import tqdm, trange

def loadData():
    df = pd.read_csv('/content/drive/MyDrive/Hackerland/lyrics-data.csv', engine = 'python', error_bad_lines = False)
    df = df[df['ALink'] == '/queen/']
    df = df.drop(columns=['language', 'ALink', 'SLink'])
    return df

df = loadData() 

In [5]:
class SongLyrics(Dataset):  
    def __init__(self, control, truncate = False, gpt2_type = "gpt2", max_length = 1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.lyrics = []

        for row in control:
          self.lyrics.append(torch.tensor(self.tokenizer.encode(f"<|{control}|>{row[:max_length]}<|endoftext|>")))               
        if truncate:
            self.lyrics = self.lyrics[:20000]
        self.lyrics_count = len(self.lyrics)
        
    def __len__(self):
        return self.lyrics_count

    def __getitem__(self, item):
        return self.lyrics[item]

In [6]:
dataset = SongLyrics(df['Lyric'], truncate=True, gpt2_type="gpt2")  

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [7]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

In [9]:
def train(dataset, model, tokenizer, batch_size=15, epochs=5, lr=2e-5, max_seq_len=400, warmupSteps=200, saveModel=False):
  
    device = torch.device("cuda")
    model = model.cuda()
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmupSteps, num_training_steps=-1)

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss = 0
    accumulating_batch_count = 0
    inputTensor = None

    for epoch in range(epochs):
        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (inputTensor, carry_on, remainder) = pack_tensor(entry, inputTensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            inputTensor = inputTensor.to(device)
            outputs = model(inputTensor, labels=inputTensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            inputTensor = None

    if saveModel:
      torch.save(model.state_dict(), '/content/drive/MyDrive/Hackerland/modelWeights.pt')
            
    return model

In [11]:
model = train(dataset, model, tokenizer, epochs=50, saveModel=True)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Training epoch 0
0


253it [00:22, 11.29it/s]


Training epoch 1
tensor(1.4301, device='cuda:0', grad_fn=<NllLossBackward0>)


253it [00:23, 10.97it/s]


...

253it [00:22, 11.18it/s]


Training epoch 48
tensor(0.8905, device='cuda:0', grad_fn=<NllLossBackward0>)


253it [00:22, 11.26it/s]


Training epoch 49
tensor(0.0423, device='cuda:0', grad_fn=<NllLossBackward0>)


253it [00:22, 11.25it/s]


In [21]:
def generate(model, tokenizer, prompt, entryCount=1, entryLength=75, topP=0.8, temp=1.0):

    model.eval()
    generatedList = []
    filterVal = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entryCount):

            entryFin = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entryLength):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temp if temp > 0 else 1.0)

                sortedLogits, sortedIndices = torch.sort(logits, descending=True)
                cumulativeProbs = torch.cumsum(torch.nn.functional.softmax(sortedLogits, dim=-1), dim=-1)

                sortedIndicesToRemove = cumulativeProbs > topP
                sortedIndicesToRemove[..., 1:] = sortedIndicesToRemove[
                    ..., :-1
                ].clone()
                sortedIndicesToRemove[..., 0] = 0

                indices_to_remove = sortedIndices[sortedIndicesToRemove]
                logits[:, indices_to_remove] = filterVal

                next_token = torch.multinomial(torch.nn.functional.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entryFin = True

                if entryFin:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generatedList.append(output_text)
                    break
            
            if not entryFin:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}" 
              generatedList.append(output_text)
                
    return generatedList[0]

In [25]:
generate(model.to('cpu'), tokenizer, "I miss you").replace("\n\n", " ")

100%|██████████| 1/1 [00:28<00:00, 28.51s/it]


'I miss you my friend!" To be honest I was the only one who really believed this was the real story, but the truth was I was in a rush and couldn\'t seem to stop it. When my mom sent me on a Monday to see me my sense of purpose was turning to gloom and sadness. I had just been crowned queen of the country and everything had just been thrown away. I couldn\'t take it anymore and I didn\'t want to go back to my'